In [261]:
import pandas as pd
df_path = '/Users/shin-yoonhwan/vscode/stock_data/fill_na_df.csv'
df = pd.read_csv(df_path)

In [262]:
filtered_df = df[['EPS', 'BPS', 'PER', 'PBR', 'ROE', \
       'ROA', 'ROIC', 'Debt_ratio', '매출액', '영업이익', '순이익', '자산총액', '부채총액',\
       '자본총액', '영업활동', '투자활동', '재무활동', '금리', '금리등락률', 'GDP', 'GDP등락률', '유가',\
       '유가등락률', 'S&P500', 'volume', 'Market Cap', 'index',\
       'Sector_idx', 'close']]

In [119]:
filtered_df.iloc[0][-1]

/var/folders/p5/2j4jc0b97cqdybgz49lxqzv40000gn/T/ipykernel_63509/419656710.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  filtered_df.iloc[0][-1]


np.float64(225.9)

In [97]:
data = filtered_df.drop(columns= 'close')
target = filtered_df['close']
data = data.to_numpy()
target = target.to_numpy()
data = data.reshape(486, 25, 28)
target = target.reshape(486, 25)

In [98]:
print(data.shape)
print(target.shape)

(486, 25, 28)
(486, 25)


In [233]:
temp_data = data
temp_target = target

In [245]:
data = filtered_df.drop(columns= 'close')
target = filtered_df['close']
temp_data = temp_data[:25]
temp_target = target[:25]

In [218]:
data.shape

(25, 28)

In [325]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# 예시 데이터 (가정)
num_companies = 486
num_periods = 25
num_features = 28

data = filtered_df.drop(columns= 'close')
target = filtered_df['close']
data = data.to_numpy()
target = target.to_numpy()
data = data.reshape(num_companies, num_periods, num_features)
target = target.reshape(num_companies, num_periods)

# 1. 데이터 정규화 (Min-Max Scaling)
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
data_scaled = scaler.fit_transform(data.reshape(-1, num_features)).reshape(num_companies, num_periods, num_features)

# 타겟은 주가 예측이므로 타겟 데이터만 정규화
target_scaled = scaler.fit_transform(target.reshape(-1, 1)).reshape(num_companies, num_periods)

# 2. 시계열 데이터 생성 (X: t-1 데이터, y: t 데이터)
X, y = [], []
for company_data, company_target in zip(data_scaled, target_scaled):
    for i in range(0, num_periods):  # 첫 번째 분기는 예측할 데이터가 없으므로 i=1부터 시작
        X.append(company_data[:i])  # t-1까지 데이터
        y.append(company_target[i])  # t 시점 주가
y = np.array(y)
# 3. Padding 시계열 데이터 (길이가 다른 데이터를 맞추기 위해)
X = tf.keras.preprocessing.sequence.pad_sequences(X, dtype="float32", padding="pre")

# 4. 데이터 분리
original_indices = np.arange(len(X))

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, original_indices, test_size=0.2, random_state=1)

model = Sequential([
    LSTM(50, activation='tanh', return_sequences=True, input_shape=(X.shape[1], num_features)),
    Dense(50, activation='relu'),
    LSTM(100, activation='tanh', return_sequences=True),  # 마지막 LSTM은 return_sequences=False
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.3),
    LSTM(100, activation='tanh', return_sequences=True),  # 마지막 LSTM은 return_sequences=False
    Dropout(0.2),
    LSTM(50, activation='tanh', return_sequences=False),  # 마지막 LSTM은 return_sequences=False
    Dropout(0.3),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='linear')
])


model.compile(optimizer='adam', loss='mse')
model.summary()

# 6. 모델 학습
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.3)

# 7. 예측
predictions = model.predict(X_test)

# 8. 예측 결과 디스케일링
predicted_prices = scaler.inverse_transform(predictions)
actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))
error_sum = 0
for predicted_price, actual_price, idx in zip(predicted_prices, actual_prices, test_idx):
    error = abs(predicted_price - actual_price)
    error_ratio = error/actual_price
    error_sum += error_ratio
    print(error_ratio)
    print(f"Predicted Price: {predicted_price}, Actual Price : {actual_price}, index : {df.loc[idx]['Ticker']}, Quarter : {df.loc[idx]['Quarter']}")
print(error_sum/len(predicted_prices))


/Users/shin-yoonhwan/vscode/stock_data/env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_75"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_133 (LSTM)                 │ (None, 24, 50)         │        15,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_135 (Dense)               │ (None, 24, 50)         │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_134 (LSTM)                 │ (None, 24, 100)        │        60,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_101 (Dropout)           │ (None, 24, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_136 (Dense)               │ (None, 24, 64)         │         6,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_102 (Dropout)           │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_135 (LSTM)                 │ (None, 24, 100)        │        66,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_103 (Dropout)           │ (None, 24, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_136 (LSTM)                 │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_104 (Dropout)           │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_137 (Dense)               │ (None, 100)            │         5,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_105 (Dropout)           │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_138 (Dense)               │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 186,615 (728.96 KB)

 Trainable params: 186,615 (728.96 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
213/213 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - loss: 3.3193 - val_loss: 2.9814
Epoch 2/30
213/213 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - loss: 2.6164 - val_loss: 2.4183
Epoch 3/30
213/213 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - loss: 1.0415 - val_loss: 2.7247
Epoch 4/30
213/213 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - loss: 2.3040 - val_loss: 1.8393
Epoch 5/30
213/213 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - loss: 0.7515 - val_loss: 1.8174
Epoch 6/30
213/213 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - loss: 0.9288 - val_loss: 1.4905
Epoch 7/30
213/213 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - loss: 0.5284 - val_loss: 1.4232
Epoch 8/30
213/213 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - loss: 0.4226 - val_loss: 1.4002
Epoch 9/30
213/213 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - loss: 0.6524 - val_loss: 1.3428
Epoch 10/30
213/213 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - loss: 0.5470 - val_loss: 1.3288
Epoch 11/30
213/213 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - loss: 0.4876 - val_loss: 1.3635
Epoch 12/30
213/213 ━━━━━━━━━━

In [202]:
inverse_X = scaler.inverse_transform(X.reshape(-1, 1))

In [287]:
temp_data = data[data['index'] == 0]
temp_target = target[data['index'] == 0]

In [305]:
data = filtered_df.drop(columns= 'close')
target = filtered_df['close']
temp_data = data[data['index'] == 0]
temp_target = target[data['index'] == 0]

In [306]:
temp_target.shape

(25,)

In [307]:

# 예시 데이터 (가정)
num_companies = 1
num_periods = 25
num_features = 28

temp_data_np = temp_data.to_numpy()
temp_target_np = temp_target.to_numpy()
temp_data_np = temp_data_np.reshape(25, 28)
temp_target_np = temp_target_np.reshape(25)

# 1. 데이터 정규화 (Min-Max Scaling)
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
data_scaled = scaler.fit_transform(temp_data_np.reshape(-1, num_features)).reshape(num_periods, num_features)

# 타겟은 주가 예측이므로 타겟 데이터만 정규화
target_scaled = scaler.fit_transform(temp_target_np.reshape(-1, 1)).reshape(num_periods)

# 2. 시계열 데이터 생성 (X: t-1 데이터, y: t 데이터)
X, y = [], []
# for company_data, company_target in zip(data_scaled, target_scaled):
for i in range(0, num_periods):  # 첫 번째 분기는 예측할 데이터가 없으므로 i=1부터 시작
    X.append(data_scaled[:i])  # t-1까지 데이터
    y.append(target_scaled[i])  # t 시점 주가
y = np.array(y)
# 3. Padding 시계열 데이터 (길이가 다른 데이터를 맞추기 위해)
X = tf.keras.preprocessing.sequence.pad_sequences(X, dtype="float32", padding="pre")

In [308]:
predict = model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [297]:
predict_inverse = scaler.inverse_transform(predict)

In [309]:
predict_inverse

array([[227.40776 ],
       [296.5855  ],
       [227.73128 ],
       [189.51247 ],
       [203.32047 ],
       [203.32285 ],
       [223.07956 ],
       [194.02713 ],
       [182.24368 ],
       [165.98528 ],
       [156.63734 ],
       [171.07707 ],
       [181.80295 ],
       [173.65555 ],
       [154.03082 ],
       [144.03462 ],
       [138.94196 ],
       [134.38882 ],
       [127.78194 ],
       [122.65644 ],
       [115.86793 ],
       [110.9063  ],
       [108.6341  ],
       [109.202934],
       [111.54781 ]], dtype=float32)